<a href="https://colab.research.google.com/github/tinkvu/ResumeRoaster/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
pip install PyPDF2 python-docx gTTS groq pydub

In [ ]:
# Import necessary libraries
import PyPDF2
import docx
from gtts import gTTS
import os
from groq import Groq

# Function to extract text from a PDF resume
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to extract text from a DOCX resume
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

# Function to call Groq Llama model for generating the HR conversation
def generate_hr_conversation(resume_text, job_role):
    # Create a client instance for Groq
    client = Groq(
        api_key="",
    )

    # Define the prompt to pass to Llama3-8b-8192 model
    prompt = f"""
    You're two senior HR executives evaluating a candidate's resume for the role of {job_role}.
    Here is the candidate's resume:

    {resume_text}

    Now, start a humorous conversation between 2 HRs talking to eachother where you provide critical feedback, roast the resume, and suggest improvements.
    -Do not use markdown, emojis, or other formatting in your responses. Respond in a way easily spoken by text-to-speech software
    -Do not write anything in braces
    -The output should be like - "HR1: <text>\nHR2: <text>\n...."
    """

    # Call the Groq API for chat completion
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None
    )

    # Gather the output as chunks of text (streaming mode)
    hr_conversation = ""
    for chunk in completion:
        hr_conversation += chunk.choices[0].delta.content or ""

    return hr_conversation

# Function to convert text to audio (using gTTS)
def text_to_speech(text, output_file="hr_feedback.mp3"):
    tts = gTTS(text, lang='en', tld='co.in')
    tts.save(output_file)
    print(f"Audio feedback saved as {output_file}")

# Main function to process the resume and provide feedback using Groq
def process_resume_with_groq(resume_path, job_role):
    # Determine file type and extract text
    if resume_path.endswith('.pdf'):
        resume_text = extract_text_from_pdf(resume_path)
    elif resume_path.endswith('.docx'):
        resume_text = extract_text_from_docx(resume_path)
    else:
        return "Unsupported file format. Please upload a PDF or DOCX file."

    # Generate HR conversation using Groq's Llama model
    hr_feedback = generate_hr_conversation(resume_text, job_role)

    # Convert the HR feedback to speech
    text_to_speech(hr_feedback)

    return hr_feedback

# Example usage
resume_path = '/content/Rinshad-DataScientist.pdf'  # Path to the uploaded resume
job_role = 'Data Scientist'  # Example job role

# Call the function to process the resume and generate feedback
feedback = process_resume_with_groq(resume_path, job_role)
print(feedback)


Audio feedback saved as hr_feedback.mp3
HR1: So, what do you think of this rogue's gallery of projects and skills?

HR2: Ha! I think it's a Frankenstein's monster of a resume. There's so much going on, it's hard to know what's the main event. And what's with all these acronyms? Are they trying to win some kind of bet?

HR1: I know, right? It's like they're trying to cram as much as possible into one page. And the font is huge in some places. Did they use a font size chart or just eyeball it?

HR2: Yeah, and the layout is all wonky. It's like a game of resume limbo – how low can you go?

HR1: Alright, so let's talk about the experience. What do we really learn from these long lists of responsibilities?

HR2: Not much, sadly. It's all just a bunch of jargon and buzzwords. Where are the specific accomplishments? Concrete numbers? Anything that makes me go "wow"?

HR1: Exactly. And what's with the personal projects? Are they trying to make us think they're a jack-of-all-trades or something

#Try with using different voice for both HRs

In [ ]:
pip install pydub

In [31]:
# Import necessary libraries
import PyPDF2
import docx
from gtts import gTTS
from pydub import AudioSegment
import os
from groq import Groq
import time
from IPython.display import Audio

# Function to extract text from a PDF resume
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                text += page.extract_text()
        return text if text else "Unable to extract text from PDF."
    except Exception as e:
        return f"Error extracting text from PDF: {e}"

# Function to extract text from a DOCX resume
def extract_text_from_docx(docx_path):
    try:
        doc = docx.Document(docx_path)
        text = '\n'.join([para.text for para in doc.paragraphs])
        return text if text else "Unable to extract text from DOCX."
    except Exception as e:
        return f"Error extracting text from DOCX: {e}"

# Function to call Groq Llama model for generating the HR conversation
def generate_hr_conversation(resume_text, job_role, job_level):
    try:
        # Create a client instance for Groq
        client = Groq(
            api_key="",
        )

        # Define the prompt to pass to Llama3-8b-8192 model
        prompt = f"""
        You're two senior HR executives evaluating a candidate's resume for the role of {job_role}. It is a {job_level} level job.
        Here is the candidate's resume:

        {resume_text}

        Now, start a humorous conversation between 2 HRs talking to each other where you provide critical feedback, roast the resume, and suggest improvements.
        -Make the conversations in a communicative tone
        -Do not use markdown, emojis, or other formatting in your responses. Respond in a way easily spoken by text-to-speech software
        -Do not write anything in braces
        -Do not use any emotions like (laughs) in your responses
        -The output should be like - "HR1: <text>\nHR2: <text>\n...."
        -Do not forget to suggest the improvements
        """

        # Call the Groq API for chat completion
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None
        )

        # Gather the output as chunks of text (streaming mode)
        hr_conversation = ""
        for chunk in completion:
            hr_conversation += chunk.choices[0].delta.content or ""

        return hr_conversation
    except Exception as e:
        return f"Error generating HR conversation: {e}"

# Function to convert HR text to audio using different voices for HR1 and HR2
def text_to_speech_mixed(hr_feedback, output_file="hr_feedback_combined.mp3"):
    # Split the feedback into lines
    lines = hr_feedback.split('\n')

    # Initialize an empty AudioSegment to store the combined audio
    combined_audio = AudioSegment.empty()

    # Loop through each line, generate audio based on HR1 or HR2, and concatenate
    for line in lines:
        if line.startswith("HR1:"):
            hr1_text = line.replace("HR1:", "").strip()
            if hr1_text:
                hr1_tts = gTTS(hr1_text, lang='en', slow=False)  # Default TTS for HR1
                hr1_tts.save("hr1_temp.mp3")
                hr1_audio = AudioSegment.from_mp3("hr1_temp.mp3")
                combined_audio += hr1_audio
        elif line.startswith("HR2:"):
            hr2_text = line.replace("HR2:", "").strip()
            if hr2_text:
                hr2_tts = gTTS(hr2_text, lang='en', tld='co.in', slow=False)  # Indian accent TTS for HR2
                hr2_tts.save("hr2_temp.mp3")
                hr2_audio = AudioSegment.from_mp3("hr2_temp.mp3")
                combined_audio += hr2_audio

    # Save the combined audio as a single output file
    combined_audio.export(output_file, format="mp3")
    print(f"Combined audio feedback saved as {output_file}")

    # Clean up temporary audio files
    if os.path.exists("hr1_temp.mp3"):
        os.remove("hr1_temp.mp3")
    if os.path.exists("hr2_temp.mp3"):
        os.remove("hr2_temp.mp3")

# Main function to process the resume and provide feedback using Groq
def process_resume_with_groq(resume_path, job_role, job_level):
    # Determine file type and extract text
    if resume_path.endswith('.pdf'):
        resume_text = extract_text_from_pdf(resume_path)
    elif resume_path.endswith('.docx'):
        resume_text = extract_text_from_docx(resume_path)
    else:
        return "Unsupported file format. Please upload a PDF or DOCX file."

    if "Error" in resume_text:
        return resume_text  # Return the error message if extraction failed

    # Generate HR conversation using Groq's Llama model
    hr_feedback = generate_hr_conversation(resume_text, job_role, job_level)

    if "Error" in hr_feedback:
        return hr_feedback  # Return the error message if generation failed

    # Convert the HR feedback to speech with alternating voices
    text_to_speech_mixed(hr_feedback)

    return hr_feedback

# Example usage
resume_path = '/content/Rinshad-DataScientist.pdf'  # Path to the uploaded resume
job_role = 'Data Scientist'  # Example job role
job_level='Beginner'

start=time.time()
# Call the function to process the resume and generate feedback
feedback = process_resume_with_groq(resume_path, job_role, job_level)
print(feedback)
end=time.time()
print("time taken for feedback generation: ", end-start)
# Display an audio player
Audio("/content/hr_feedback_combined.mp3", autoplay=True)

Combined audio feedback saved as hr_feedback_combined.mp3
HR1: Well, it looks like Rinshad has a lot of enthusiasm for AI and machine learning, but I'm not sure if his resume does it justice.

HR2: Yeah, it's a bit of a mess. I mean, he has some big words like "Efficient use of data" and "Best use cases with the data", but how many times does he use that phrase? It's like he's trying to jam-pack every possible buzzword into a single bullet point.

HR1: Exactly. And have you seen the font size on this thing? It's like he's trying to blind us with science.

HR2: (chuckles) Yeah, maybe that's what he's going for. But seriously, Rinshad does seem to have some experience working with data science and AI-related projects. I'll give him credit for that.

HR1: And I'll give him credit for having a variety of projects to his name. But some of them, like "Math Symbol Handwriting Classification using CNN" seem a bit... niche.

HR2: Niche? That's one way to put it. I think "redundant" might be a b

In [ ]:
from gtts import gTTS
from IPython.display import Audio

# Create and save the TTS output
test = gTTS("Hello, How are you doing? It is a nice day today", lang='en', tld='ca', slow=False)
test.save("test1.mp3")

# Display an audio player
Audio("test1.mp3", autoplay=True)


In [ ]:
from gtts import gTTS
from IPython.display import Audio

# Create and save the TTS output
test = gTTS("Hello, How are you doing? It is a nice day today", lang='en', tld='ie', slow=False)
test.save("test.mp3")

# Display an audio player
Audio("test.mp3", autoplay=True)
